전문가를 위한 파이썬
========
3-1 일반적인 맵핑형
--------------

1. Key 값은 Hash해야 한다.
2. dict, UserDict 때문에 isinstance를 이용를 검사하는게 좋다. 다른 맵핑형이 사용될 수 있기 때문이다.

In [1]:
my_dict = {}
import collections
isinstance(my_dict, collections.abc.Mapping)

True

#### Dict의 비교
1. dict
2. defaultdict
3. OrderedDict

dict.get('key', default) 는 사용 안하는게 좋다.

##### setdefault()를 이용하면 코드 길이가 줄어든다.

In [13]:
key = 'key'
new_value = 'value'

In [12]:
my_dict = {}
my_dict.setdefault(key, []).append(new_value)

print(my_dict)

{'key': ['value']}


In [11]:
my_dict = {}
if key not in my_dict:
    my_dict[key] = []
my_dict[key].append(new_value)

print(my_dict)

{'key': ['value']}


###### defaultdict은 아래와 같은 구조를 가진다.

In [6]:
class defaultdict(dict):
    def __init__(self, default_factory=None, **kwargs):
        """
        :type default_factory: () -> V
        :rtype: defaultdict[Any, V]
        """
        pass

    def __missing__(self, key):
        """
        :type key: Any
        :rtype: V
        """
        pass

##### 그래서 아래와 같이 구현하면, default_factory는 list가 된다.

In [9]:
import collections
index = collections.defaultdict(list)

##### dict[number] 를 호출하면 __getitem__() 메타 메소드가 호출된다. 여기서 key 값이 없다면 __missing__() 메타 메소드가 호출된다.

##### 기본적으로 mapping을 상속하여 사용하는 경우는 UserDict을 사용하는게 좋다.
+ __missing__과 __contains__ 재귀적인 호출을 피하기 위해 isinstance()와 key in self.keys()와 같이 명시적으로 정의하였다.
+ key in dict 보단 key in dict.keys()가 성능이 좋다.

In [1]:
class StrKeyDict0(dict):

    def __missing__(self, key):
        if isinstance(key, str):
            raise KeyError(key)
        return self[str(key)]

    def get(self, key, default=None):
        try:
            return self[key]
        except KeyError:
            return default

    def __contains__(self, key):
        return key in self.keys() or str(key) in self.keys()

### 3-5 그외 맵핑형
* collections.OrderDict: 키를 삽입한 순서대로(마지막) 사용이 가능하다 popitem(last=True)
* collections.ChainMap:.....????
* Collections.Counter

In [7]:
import builtins
import collections
pylookup = collections.ChainMap(locals(), globals(), vars(builtins))
print(pylookup)

ChainMap({'__name__': '__main__', '__doc__': 'Automatically created module for IPython interactive environment', '__package__': None, '__loader__': None, '__spec__': None, '__builtin__': <module 'builtins' (built-in)>, '__builtins__': <module 'builtins' (built-in)>, '_ih': ['', 'class StrKeyDict0(dict):\n\n    def __missing__(self, key):\n        if isinstance(key, str):\n            raise KeyError(key)\n        return self[str(key)]\n\n    def get(self, key, default=None):\n        try:\n            return self[key]\n        except KeyError:\n            return default\n\n    def __contains__(self, key):\n        return key in self.keys() or str(key) in self.keys()', "import collections\nct = collections.Counter('abracadabra')\nprint(ct)", 'import builtins\npylookup = Chainlt/lap(localsO, globalsO, vars(builtins))\nprint(pylookup)', 'import builtins\npylookup = ChainMap(locals(), globals(), vars(builtins))\nprint(pylookup)', 'import builtins\nimport collections.ChainMap\npylookup = Ch

In [2]:
import collections
ct = collections.Counter('abracadabra')
print(ct)

Counter({'a': 5, 'b': 2, 'r': 2, 'c': 1, 'd': 1})


### 3-6 UserDict 상속하기

* UserDict을 상속하면 좋은 이유는 다음과 같다.
    1. data라는 dict 객체를 갖고있다. 그래서 __setitem__에 재귀호출, __contains__ 메서드를 간단히 구현할 수 있다.


In [9]:
import collections
class StrKeyDict(collections.UserDict):
    def _missing_(self , key):
        if isinstance(key, str):
            raise KeyError(key) 
        return selfIstr(key)
    def _contains_(self , key):
        return str(key) in self.data
    def _setitem_(self, key, item):
        self.data[str(key)] = item

### 3-7 불변 매핑
* MappingProxyType 이라는 래퍼 클래스를 이용하여, 기존 매핑은 반영이 되나, 직접 변경할 수 는 없는 매핑객체를 만들 수 있다.

In [12]:
from types import MappingProxyType
d={1:'A'}
d_proxy = MappingProxyType(d)
d_proxy

mappingproxy({1: 'A'})

In [13]:
d_proxy[1]

'A'

In [14]:
d_proxy[2] = 'B'

TypeError: 'mappingproxy' object does not support item assignment

In [15]:
d[2] = 'B'
d_proxy

mappingproxy({1: 'A', 2: 'B'})

### 3-8 집합 이론

* 집합 연산을 이용하자 & | - 와 같은... 가독성 및 성능이 뛰어나다.

In [18]:
from dis import dis

dis('{1}')

  1           0 LOAD_CONST               0 (1)
              2 BUILD_SET                1
              4 RETURN_VALUE


In [17]:
dis('set([1])')

  1           0 LOAD_NAME                0 (set)
              2 LOAD_CONST               0 (1)
              4 BUILD_LIST               1
              6 CALL_FUNCTION            1
              8 RETURN_VALUE


#### dis 함수를 이용하여 리터럴 표현식{1}과 set([1])에 작업 단계를 비교하면 {1}로 집합을 생성하는게 효율적인 것을 알 수 있다.
> {1}는 BUILD_SET 이라는 특수 바이트코드가 모든 일을 처리한다.